In [1]:
import os
import time
from argparse import ArgumentParser

import pandas as pd
import numpy as np
import psycopg2

In [2]:
CREATE_LOAD_SITE = """
CREATE TABLE IF NOT EXISTS load_site (
    id SERIAL PRIMARY KEY,
    load_code TEXT UNIQUE NOT NULL,
    location TEXT,
    description TEXT
);
"""

In [3]:
CREATE_SOLAR_SITE = """
CREATE TABLE IF NOT EXISTS solar_site (
    id SERIAL PRIMARY KEY,
    solar_code TEXT UNIQUE NOT NULL,
    load_id INTEGER NOT NULL,
    panel_capacity_kw FLOAT,
    FOREIGN KEY (load_id) REFERENCES load_site(id)
);
"""

In [4]:
CREATE_LOAD_DATA = """
CREATE TABLE IF NOT EXISTS load_data (
    id SERIAL PRIMARY KEY,
    timestamp TIMESTAMP NOT NULL,
    load_id INTEGER NOT NULL,
    demand FLOAT,
    FOREIGN KEY (load_id) REFERENCES load_site(id),
    UNIQUE (timestamp, load_id)
);
"""

CREATE_SOLAR_DATA= """
CREATE TABLE IF NOT EXISTS solar_data (
    id SERIAL PRIMARY KEY,
    timestamp TIMESTAMP NOT NULL,
    solar_id INTEGER NOT NULL,
    generation FLOAT,
    FOREIGN KEY (solar_id) REFERENCES solar_site(id),
    UNIQUE (timestamp, solar_id)
);
"""

In [5]:
CREATE_LOAD_WEATHER = """
CREATE TABLE IF NOT EXISTS load_weather (
    id SERIAL PRIMARY KEY,
    timestamp TIMESTAMP NOT NULL,
    load_id INTEGER NOT NULL,
    temperature FLOAT,
    humidity FLOAT,
    FOREIGN KEY (load_id) REFERENCES load_site(id),
    UNIQUE (timestamp, load_id)
);
"""

CREATE_SOLAR_WEATHER = """
CREATE TABLE IF NOT EXISTS solar_weather (
    id SERIAL PRIMARY KEY,
    timestamp TIMESTAMP NOT NULL,
    solar_id INTEGER NOT NULL,
    temperature FLOAT,
    humidity FLOAT,
    wind_speed FLOAT,
    wind_direction FLOAT,
    cloud_cover VARCHAR(20),
    FOREIGN KEY (solar_id) REFERENCES solar_site(id),
    UNIQUE (timestamp, solar_id)
);
"""

In [6]:
db_connect = psycopg2.connect(
    user="admin",
    password="1234",
    host='localhost',
    port=5432,
    database="machinedb",
)

In [7]:
with db_connect.cursor() as cur:
    cur.execute(CREATE_LOAD_SITE)
    cur.execute(CREATE_SOLAR_SITE)
    db_connect.commit()

In [8]:
with db_connect.cursor() as cur:
    cur.execute(CREATE_LOAD_DATA)
    cur.execute(CREATE_SOLAR_DATA)
    db_connect.commit()

In [9]:
with db_connect.cursor() as cur:
    cur.execute(CREATE_LOAD_WEATHER)
    cur.execute(CREATE_SOLAR_WEATHER)
    db_connect.commit()

In [10]:
INSERT_LOAD_SITE = """INSERT INTO load_site (load_code, location, description) VALUES
('LOAD_001', 'GWANGJU', 'factory');"""

INSERT_SOLAR_SITE = """INSERT INTO solar_site (solar_code, load_id, panel_capacity_kw) VALUES
('SOLAR_001', 1, 500),
('SOLAR_002', 1, 500),
('SOLAR_003', 1, 500),
('SOLAR_004', 1, 500),
('SOLAR_005', 1, 500);"""

with db_connect.cursor() as cur:
    cur.execute(INSERT_LOAD_SITE)
    cur.execute(INSERT_SOLAR_SITE)
    db_connect.commit()

In [11]:
import sys
sys.path.append("/home/moon/project/prediction-project")

In [12]:
load = pd.read_csv("../data/raw/load/load.csv")
load_weather = pd.read_csv("../data/raw/load/load_weather.csv")
solar = pd.read_csv("../data/raw/solar/solars.csv")
solar_weather = pd.read_csv("../data/raw/solar/weather.csv")

In [37]:
solar["datetime"] = pd.to_datetime(solar["datetime"])
solar_weather["datetime"] = pd.to_datetime(solar_weather["datetime"])
solar_df = pd.merge(solar, solar_weather, on="datetime", how="left")
solar_df.isnull().sum()

datetime           0
id                 0
power              0
temperature       20
wind_speed         5
wind_direction     5
humidity           5
cloud              0
dtype: int64

In [38]:
solar_df.ffill(inplace=True)

In [39]:
solar_df.isnull().sum()

datetime          0
id                0
power             0
temperature       0
wind_speed        0
wind_direction    0
humidity          0
cloud             0
dtype: int64

In [16]:
load_df = pd.concat([load, load_weather], axis=1)
solar_df = pd.concat([solar, solar_weather], axis=1)

In [17]:
load_df.head()

,id,load,datetime,temperature,humidity
0,1,2456.0425,2016-01-01 0:00,3.0,60.6
1,1,2570.6650,2016-01-01 1:00,2.6,60.8
2,1,2389.2700,2016-01-01 2:00,2.6,59.6
3,1,2330.2925,2016-01-01 3:00,2.1,62.4
4,1,2244.0450,2016-01-01 4:00,2.0,62.4


In [18]:
solar_df.head()

,datetime,id,power,datetime,temperature,wind_speed,wind_direction,humidity,cloud
0,2018-03-01 00:00:00,1,0.0,2018-03-01 00:00:00,8.2,3.9,340.0,98.0,Mostly
1,2018-03-01 01:00:00,1,0.0,2018-03-01 01:00:00,7.0,4.1,320.0,97.0,Mostly
2,2018-03-01 02:00:00,1,0.0,2018-03-01 02:00:00,6.5,5.9,290.0,80.0,Mostly
3,2018-03-01 03:00:00,1,0.0,2018-03-01 03:00:00,6.2,4.6,320.0,79.0,Clear
4,2018-03-01 04:00:00,1,0.0,2018-03-01 04:00:00,6.7,4.5,320.0,73.0,Clear


In [19]:
def df_to_insert_sql(df, table_name, column_order=None):
    """ DataFrame을 INSERT INTO SQL 문자열로 변환 """
    if column_order:
        df = df[column_order]

    values = []
    for row in df.itertuples(index=False, name=None):
        formatted_row = []
        for item in row:
            if pd.isna(item):
                formatted_row.append('NULL')
            elif isinstance(item, str):
                formatted_row.append(f"'{item}'")
            elif isinstance(item, pd.Timestamp):
                formatted_row.append(f"'{item.strftime('%Y-%m-%d %H:%M:%S')}'")
            else:
                formatted_row.append(str(item))
        values.append(f"({', '.join(formatted_row)})")

    columns = column_order if column_order else df.columns
    sql = f"INSERT INTO {table_name} ({', '.join(columns)}) VALUES\n" + ",\n".join(values) + ";"
    return sql

In [20]:
load_df.rename(columns={"id": "load_id", "load": "demand", "datetime": "timestamp"}, inplace=True)

In [21]:
load_df["timestamp"] = pd.to_datetime(load_df["timestamp"])

In [23]:
load_df.shape

(8784, 5)

In [30]:

load_df["timestamp"] = pd.date_range(start="2024-05-18", end="2025-05-19", freq="h")[:-1]

In [73]:
load_df[["timestamp", "load_id", "temperature", "humidity", "demand"]].to_csv("../data/raw/load/load_data.csv", index=False)

In [31]:
insert_sql = df_to_insert_sql(load_df[["timestamp", "load_id", "demand"]], "load_data", ["timestamp", "load_id", "demand"])

In [32]:
try:
    with db_connect.cursor() as cur:
        cur.execute(insert_sql)
        db_connect.commit()
except Exception as e:
    db_connect.rollback()
    print("[ERROR] SQL 실행 중 오류 발생:", e)
    print("실행 SQL:\n", insert_sql[:500], "...")  # 너무 길면 일부만 출력

In [33]:
insert_sql = df_to_insert_sql(load_df[["timestamp", "load_id", "temperature", "humidity"]], "load_weather", ["timestamp", "load_id", "temperature", "humidity"])

In [34]:
try:
    with db_connect.cursor() as cur:
        cur.execute(insert_sql)
        db_connect.commit()
except Exception as e:
    db_connect.rollback()
    print("[ERROR] SQL 실행 중 오류 발생:", e)
    print("실행 SQL:\n", insert_sql[:500], "...")  # 너무 길면 일부만 출력

In [62]:
date_range = pd.date_range(start="2024-03-01", end="2025-05-19", freq="h")[:-1]

In [99]:
solar_1 = solar_df[solar_df["id"] == 1].query("datetime > '2019-03-01 00:00:00' and datetime < '2020-05-18 23:00:00'")[:-22]
solar_1["datetime"] = date_range

solar_2 = solar_df[solar_df["id"] == 2].query("datetime > '2019-03-01 00:00:00' and datetime < '2020-05-18 23:00:00'")[:-22]
solar_2["datetime"] = date_range

solar_3 = solar_df[solar_df["id"] == 3].query("datetime > '2019-03-01 00:00:00' and datetime < '2020-05-18 23:00:00'")[:-22]
solar_3["datetime"] = date_range

solar_4 = solar_df[solar_df["id"] == 4].query("datetime > '2019-03-01 00:00:00' and datetime < '2020-05-18 23:00:00'")[:-22]
solar_4["datetime"] = date_range

solar_5 = solar_df[solar_df["id"] == 5].query("datetime > '2019-03-01 00:00:00' and datetime < '2020-05-18 23:00:00'")[:-22]
solar_5["datetime"] = date_range

In [100]:
solar_1.rename(columns={"id": "solar_id", "power": "generation", "datetime": "timestamp", "cloud": "cloud_cover"}, inplace=True)
solar_2.rename(columns={"id": "solar_id", "power": "generation", "datetime": "timestamp", "cloud": "cloud_cover"}, inplace=True)
solar_3.rename(columns={"id": "solar_id", "power": "generation", "datetime": "timestamp", "cloud": "cloud_cover"}, inplace=True)
solar_4.rename(columns={"id": "solar_id", "power": "generation", "datetime": "timestamp", "cloud": "cloud_cover"}, inplace=True)
solar_5.rename(columns={"id": "solar_id", "power": "generation", "datetime": "timestamp", "cloud": "cloud_cover"}, inplace=True)

In [101]:
pd.concat([solar_1, solar_2, solar_3, solar_4, solar_5], axis=0).reset_index(drop=True).to_csv("../data/raw/solar/solar_data.csv", index=False)

In [ ]:
insert_sql = df_to_insert_sql(solar_df[["timestamp", "solar_id", "generation"]], "solar_data", ["timestamp", "solar_id", "generation"])

In [71]:
try:
    with db_connect.cursor() as cur:
        cur.execute(insert_sql)
        db_connect.commit()
except Exception as e:
    db_connect.rollback()
    print("[ERROR] SQL 실행 중 오류 발생:", e)
    print("실행 SQL:\n", insert_sql[:500], "...")  # 너무 길면 일부만 출력

In [75]:
insert_sql = df_to_insert_sql(
    solar_df[["timestamp", "solar_id", "temperature", "humidity","wind_speed", "wind_direction", "cloud_cover"]], 
    "solar_weather", 
    ["timestamp", "solar_id", "temperature", "humidity","wind_speed", "wind_direction", "cloud_cover"])

In [76]:
try:
    with db_connect.cursor() as cur:
        cur.execute(insert_sql)
        db_connect.commit()
except Exception as e:
    db_connect.rollback()
    print("[ERROR] SQL 실행 중 오류 발생:", e)
    print("실행 SQL:\n", insert_sql[:500], "...")  # 너무 길면 일부만 출력

In [77]:
def drop_table_if_exists(table_name):
    """ 테이블이 존재하면 삭제 """
    with db_connect.cursor() as cur:
        cur.execute(f"DROP TABLE IF EXISTS {table_name};")
        db_connect.commit()

In [79]:
drop_table_if_exists("load_data")
drop_table_if_exists("load_weather")
drop_table_if_exists("solar_data")
drop_table_if_exists("solar_weather")
drop_table_if_exists("solar_site")
drop_table_if_exists("load_site")


In [102]:
solar_data = pd.read_csv("../data/raw/solar/solar_data.csv")
load_data = pd.read_csv("../data/raw/load/load_data.csv")

In [ ]:
def insert_load(load_data):
    """ load_data DataFrame을 PostgreSQL에 삽입 """
    insert_sql = df_to_insert_sql(load_data, "load_data", ["timestamp", "load_id", "demand"])
    try:
        with db_connect.cursor() as cur:
            cur.execute(insert_sql)
            db_connect.commit()
    except Exception as e:
        db_connect.rollback()
        print("[ERROR] SQL 실행 중 오류 발생:", e)
        print("실행 SQL:\n", insert_sql[:500], "...")  # 너무 길면 일부만 출력
def insert_load_weather(load_data):
    """ load_weather DataFrame을 PostgreSQL에 삽입 """
    insert_sql = df_to_insert_sql(load_data, "load_weather", ["timestamp", "load_id", "temperature", "humidity"])
    try:
        with db_connect.cursor() as cur:
            cur.execute(insert_sql)
            db_connect.commit()
    except Exception as e:
        db_connect.rollback()
        print("[ERROR] SQL 실행 중 오류 발생:", e)
        print("실행 SQL:\n", insert_sql[:500], "...")  # 너무 길면 일부만 출력
        
def insert_solar(solar_data):
    """ solar_data DataFrame을 PostgreSQL에 삽입 """
    insert_sql = df_to_insert_sql(solar_data, "solar_data", ["timestamp", "solar_id", "generation"])
    try:
        with db_connect.cursor() as cur:
            cur.execute(insert_sql)
            db_connect.commit()
    except Exception as e:
        db_connect.rollback()
        print("[ERROR] SQL 실행 중 오류 발생:", e)
        print("실행 SQL:\n", insert_sql[:500], "...")  # 너무 길면 일부만 출력
        
def insert_solar_weather(solar_data):
    """ solar_weather DataFrame을 PostgreSQL에 삽입 """
    insert_sql = df_to_insert_sql(solar_data, "solar_weather", ["timestamp", "solar_id", "temperature", "humidity", "wind_speed", "wind_direction", "cloud_cover"])
    try:
        with db_connect.cursor() as cur:
            cur.execute(insert_sql)
            db_connect.commit()
    except Exception as e:
        db_connect.rollback()
        print("[ERROR] SQL 실행 중 오류 발생:", e)
        print("실행 SQL:\n", insert_sql[:500], "...")  # 너무 길면 일부만 출력

In [117]:
insert_sql = df_to_insert_sql(solar_data, "solar_data", ["timestamp", "solar_id", "generation"])
try:
    with db_connect.cursor() as cur:
        cur.execute(insert_sql)
        db_connect.commit()
except Exception as e:
    db_connect.rollback()
    print("[ERROR] SQL 실행 중 오류 발생:", e)
    print("실행 SQL:\n", insert_sql[:500], "...")  # 너무 길면 일부만 출력

In [118]:
insert_sql = df_to_insert_sql(
    solar_data[["timestamp", "solar_id", "temperature", "humidity","wind_speed", "wind_direction", "cloud_cover"]], 
    "solar_weather", 
    ["timestamp", "solar_id", "temperature", "humidity","wind_speed", "wind_direction", "cloud_cover"])
try:
    with db_connect.cursor() as cur:
        cur.execute(insert_sql)
        db_connect.commit()
except Exception as e:
    db_connect.rollback()
    print("[ERROR] SQL 실행 중 오류 발생:", e)
    print("실행 SQL:\n", insert_sql[:500], "...")  # 너무 길면 일부만 출력

In [ ]:
from datetime import datetime, timedelta

In [113]:
current_hour = datetime.now().hour
solar_1_sample = solar_data[(pd.to_datetime(solar_data["timestamp"]).dt.hour == current_hour) & (solar_data["solar_id"] == 1)].sample(1)
solar_1_sample["timestamp"] = pd.Timestamp.now().strftime('%Y-%m-%d %H:00:00')

solar_2_sample = solar_data[(pd.to_datetime(solar_data["timestamp"]).dt.hour == current_hour) & (solar_data["solar_id"] == 2)].sample(1)
solar_2_sample["timestamp"] = pd.Timestamp.now().strftime('%Y-%m-%d %H:00:00')

solar_3_sample = solar_data[(pd.to_datetime(solar_data["timestamp"]).dt.hour == current_hour) & (solar_data["solar_id"] == 3)].sample(1)
solar_3_sample["timestamp"] = pd.Timestamp.now().strftime('%Y-%m-%d %H:00:00')

solar_4_sample = solar_data[(pd.to_datetime(solar_data["timestamp"]).dt.hour == current_hour) & (solar_data["solar_id"] == 4)].sample(1)
solar_4_sample["timestamp"] = pd.Timestamp.now().strftime('%Y-%m-%d %H:00:00')

solar_5_sample = solar_data[(pd.to_datetime(solar_data["timestamp"]).dt.hour == current_hour) & (solar_data["solar_id"] == 5)].sample(1)
solar_5_sample["timestamp"] = pd.Timestamp.now().strftime('%Y-%m-%d %H:00:00')


In [116]:
insert_sql = df_to_insert_sql(solar_1_sample, "solar_data", ["timestamp", "solar_id", "generation"])
try:
    with db_connect.cursor() as cur:
        cur.execute(insert_sql)
        db_connect.commit()
except Exception as e:
    db_connect.rollback()
    print("[ERROR] SQL 실행 중 오류 발생:", e)
    print("실행 SQL:\n", insert_sql, "...")  # 너무 길면 일부만 출력